In [ ]:
import subprocess
smi=subprocess.run(["nvidia-smi", "-L"], stdout=subprocess.PIPE)
if smi.returncode != 0:
  assert False, "No GPU available!"
print(smi.stdout.decode())

GPU 0: Tesla T4 (UUID: GPU-41ed80c3-5105-8f43-1108-9ed46036dd2c)



この記事では、Google Colaboratoryにカタゴを設定し、lizzie、sabaki（または他のGTPエンジンをサポートするツール）と接続する方法について説明します。

ステップ1：ColabサーバーでKataGoとSSHを実行する
初めて実行する前に、NGROK_TOKENの値を入力してください。

Googleアカウントによって値が異なるため、コラボを実行しているコンピュータのGoogleアカウントが変わった場合は、再入力する必要があります。

下のリンクを押して移動し、一番上のコピーボタンを押すとトークン値がコピーされます。

https://dashboard.ngrok.com/auth/your-authtoken.

次に、下のスクリプト領域の一番上の行の入力場所に貼り付けてください。

USER_PASSWORDの値を希望の値に置き換えてください。

（この場合、後でPCのlizzie設定でも同じように変更しなければなりません⇒最後に出てくる命令ガイドを参照）

TeslaT4 GPUが接続されている場合に60ブロックの重みを使用するには、T4_WEIGHT値を60bに変更するだけです（推奨されません）。

次に、下から実行ボタン（丸いアイコン）を押して実行します。

そして、Googleドライブを接続するための権限取得手順があります。途中で出てくる案内に従って実行してください。

スクリプト全体が実行されるまでに1〜2分かかります。参考までに、サーバーに自動的に割り当てられたGPUがCUDAをサポートしていない場合は、最初のlizzieを実行したときにOpenCLチューニング手順が続くことがあります。

CUDA バージョンが選択された場合、およそ NVIDIA RTX 2060 程度の性能が出るようです。

In [ ]:
# SSH configuration
NGROK_TOKEN = 'your_ngrok_token'
USER_PASSWORD = 'your_password'

# KataGo configuration
KATAGO_BACKEND = 'AUTO'
T4_WEIGHT = '40b'
ELSE_WEIGHT = '20b'

# Constant
KATAGO_COLAB_REPOSITORY_URL = 'https://github.com/matobataketoshi/katago-colab.git'
KATAGO_CONFIG_FILE = '/content/katago-colab/config/gtp_colab.cfg'
KATAGO_TUNING_DIR = '/content/katago-colab/opencltuning'
KATAGO_RESOURCE_FILE = '/content/katago-colab/colab-resource/external-resource.json'
SSH_INFO_FILE_NAME = 'colab-katago-ssh.json'
SSH_INFO_DIR = '/content/drive/MyDrive'

# Install useful stuff
!echo "必要なライブラリをインストールします。"
!apt-get update 1>/dev/null
!apt-get install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
!wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.9.1/libzip.so.5.0 -O /usr/lib/x86_64-linux-gnu/libzip.so.5
!pip install -U -q PyDrive 1>/dev/null

import subprocess
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import requests
import json
import os
from re import sub

def get_katago_resource(katago_resource_file):
    with open(katago_resource_file, 'r') as f:
        resource = json.load(f)
    return resource

def save_ssh_info(dir, file_name):
    file_path = dir + '/' + file_name
    ssh_option = create_ssh_option()
    file_id = get_file_id_in_drive(file_path)
    if file_id is None:
        file_id = upload_ssh_option_with_pydrive(file_name, ssh_option)
    else:
        ssh_option['fileId'] = file_id
        write_ssh_option(file_path, ssh_option)
    print('================================================================')
    print('SSH_INFO_GOOGLE_DRIVE_FILE_ID: {}'.format(file_id))
    print('================================================================')
    return file_id

def create_ssh_option():
    ssh_option = None
    try:
        r = requests.get('http://localhost:4040/api/tunnels')
        raw_ssh = r.json()['tunnels'][0]['public_url']
        ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
        ssh_option = {
            'host': ssh_args[0],
            'port': int(ssh_args[1]),
            'user': 'root'
        }
    except Exception as e:
        print('Failed to create SSH options. Make sure that the ngrok token is set correctly.')
        raise e
    return ssh_option

def get_file_id_in_drive(file_path):
    file_id = None
    if (os.path.exists(file_path)):
        try:
            with open(file_path, 'r') as f:
                ssh_option = json.load(f)
                file_id = ssh_option.get('fileId')
        except Exception as e:
            file_id = None
    return file_id

def write_ssh_option(file_path, ssh_option):
    print('Write SSH option: {}'.format(file_path))
    with open(file_path, 'w') as f:
        f.write(json.dumps(ssh_option))
 
def upload_ssh_option_with_pydrive(file_name, ssh_option):
    print('Authenticate to Google Drive')
    drive = auth_google_drive()
    file_metadata = {
        'title': file_name,
        'mimeType': 'application/json'
    }
    file_list = drive.ListFile({'q': 'title="' + file_name + '" and trashed=False'}).GetList()
    if (len(file_list) > 0):
        file_metadata['id'] = file_list[0]['id']
    ssh_option_file = drive.CreateFile(file_metadata)
    ssh_option_file.Upload()
    file_id = ssh_option_file.get('id')
    ssh_option['fileId'] = file_id
    ssh_option_file.SetContentString(json.dumps(ssh_option))
    ssh_option_file.Upload()
    ssh_option_file.InsertPermission({'type': 'anyone', 'value': 'anyone', 'role': 'reader'})
    return file_id

def auth_google_drive():
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    return drive

# Get GPU name
gpu_name = str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
print('GPU: {}'.format(gpu_name))

# Select backend type
if KATAGO_BACKEND == "AUTO":
  if gpu_name == "TeslaT4":
    KATAGO_BACKEND = "TensorRT"
  else:
    KATAGO_BACKEND = "OPENCL"    
print('Using KataGo Binery: {}'.format(KATAGO_BACKEND))

# Select weight file
if gpu_name == "TeslaT4":
  WEIGHT_FILE = T4_WEIGHT
else:
  WEIGHT_FILE = ELSE_WEIGHT
print('Using KataGo Weight: {}'.format(WEIGHT_FILE))

%cd /content
# Clone katago-colab
!echo "Gitから必要なファイルを受け取ります。"
!rm -rf katago-colab
!git clone $KATAGO_COLAB_REPOSITORY_URL 1>/dev/null

# Get URLs of external resources
katago_resource = get_katago_resource(KATAGO_RESOURCE_FILE)
ngrok_url = katago_resource['ngrok']
katago_url = katago_resource[KATAGO_BACKEND]
weight_url = katago_resource[WEIGHT_FILE]

if KATAGO_BACKEND == "TensorRT":
  !echo "TensorRTの駆動に必要なファイルをインストールします。"

  %cd /content
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libcudnn8_8.2.1.32-1+cuda11.3_amd64.deb
  !dpkg -i libcudnn8_8.2.1.32-1+cuda11.3_amd64.deb

  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvinfer8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvinfer8_8.2.2-1+cuda11.4_amd64.deb
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvonnxparsers8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvonnxparsers8_8.2.2-1+cuda11.4_amd64.deb
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/libnvparsers8_8.2.2-1+cuda11.4_amd64.deb
  !dpkg -i libnvparsers8_8.2.2-1+cuda11.4_amd64.deb

# Download ngrok
!wget -q $ngrok_url -O ngrok
!chmod +x /content/ngrok
 
# SSH setting
!echo "root:$USER_PASSWORD" | chpasswd
!echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
!echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

!mkdir -p /root/.ssh
!service ssh restart > /dev/null
 
# Run ngrok
!echo "ngrokを実行します。"
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
!sleep 5

# Save SSH info to Google Drive
!echo "GoogleドライブにSSH接続情報を保存します。"
SSH_INFO_GOOGLE_DRIVE_FILE_ID = save_ssh_info(SSH_INFO_DIR, SSH_INFO_FILE_NAME)

# Download KataGo binary
!echo "Katago実行ファイルを受け取ります。"
!wget -q "$katago_url" -O katago
!chmod +x /content/katago
!/content/katago version

# Put KataGo tuning files
!mkdir -p /root/.katago/
if KATAGO_BACKEND == "OPENCL":
  !cp -r $KATAGO_TUNING_DIR /root/.katago/

# Download a network file of KataGo
!echo "Download a network file of KataGo"
!rm -rf weight.bin.gz
!wget -q "$weight_url" -O "weight.bin.gz"

# Copy enviroment variable for SSH
with open('/root/.ssh/environment','w') as f:
    for key, value in os.environ.items():
        print('{}={}'.format(key, value), file=f)

!echo -e "\n[KataGo Config]" 
!cat $KATAGO_CONFIG_FILE
!echo -e "\n"

!echo "接続の準備が整いました! lizzieを接続できます..."
!echo "lizzie engine 駆動命令 : ./colab-katago-gd.exe $SSH_INFO_GOOGLE_DRIVE_FILE_ID $USER_PASSWORD"

ステップ2：Colab KataGoをsabakiまたはlizzieに接続する
（GOOGLE_DRIVE_IDとユーザーパスワードは直接編集する必要があります）
下記のリンクから配布パッケージを受け取ってインストールし、lizzieエンジン駆動命令を修正してご使用ください。

https://github.com/wonsiks/katago-colab/releases/download/v1.9.1/colab-katago-gd.exe

In [ ]:
%%writefile /content/katago-colab/config/gtp_colab.cfg

# Config for C++ (non-python) gtp bot

# RUNNING ON AN ONLINE SERVER OR IN A REAL TOURNAMENT OR MATCH:
# If you plan to do so, you may want to read through the "Rules" section
# below carefully for proper handling of komi and handicap games and end-of-game cleanup
# and various other details.

# NOTES ABOUT PERFORMANCE AND MEMORY USAGE:
# You will likely want to tune one or more the following:
#
# numSearchThreads:
# The number of CPU threads to use. If your GPU is powerful, it can actually be much higher than
# the number of cores on your processor because you will need many threads to feed large enough
# batches to make good use of the GPU.
#
# The "./katago benchmark" command can help you tune this parameter, as well as to test out the effect
# of changes to any of the other parameters below!
#
# nnCacheSizePowerOfTwo:
# This controls the NN Cache size, which is the primary RAM/memory use.
# Increase this if you don't mind the memory use and want better performance for searches with
# tens of thousands of visits or more. Decrease this if you want to limit memory usage.
#
# If you're someone who is happy to do a bit of math - each neural net entry takes very
# approximately 1.5KB, except when using whole-board ownership/territory visualizations, each
# entry will take very approximately 3KB. The number of entries is (2 ** nnCacheSizePowerOfTwo),
# for example 2 ** 18 = 262144.
#
# OTHER NOTES:
# If you have more than one GPU, take a look at "OpenCL GPU settings" or "CUDA GPU settings" below.
#
# If using OpenCL, you will want to verify that KataGo is picking up the correct device!
# (e.g. some systems may have both an Intel CPU OpenCL and GPU OpenCL, if KataGo appears to pick
# the wrong one, you correct this by specifying "openclGpuToUse" below).
#
# You may also want to adjust "maxVisits", "ponderingEnabled", "resignThreshold", and possibly
# other parameters depending on your intended usage.
#
# ----------------------------------------------------------------------------------------

# For the `katago gtp` command, ALL of THE BELOW VALUES MAY BE SET OR OVERRIDDEN if desired via
# the command line arguments:
# -override-config KEY=VALUE,KEY=VALUE,...

# Logs and files--------------------------------------------------------------------------

# Where to output log?
logDir = gtp_logs    # Each run of KataGo will log to a separate file in this dir
# logFile = gtp.log  # Use this instead of logDir to just specify a single file directly

# Logging options
logAllGTPCommunication = true
logSearchInfo = true
logToStderr = false

# KataGo will display some info to stderr on GTP startup
# Uncomment this to suppress that and remain silent
# startupPrintMessageToStderr = false

# Chat some stuff to stderr, for use in things like malkovich chat to OGS.
# ogsChatToStderr = true

# Optionally override where KataGo will attempt to save things like openCLTuner files and other cached data.
# homeDataDir = DIRECTORY

# Analysis------------------------------------------------------------------------------------

# Configure the maximum length of analysis printed out by lz-analyze and other places.
# Controls the number of moves after the first move in a variation.
# analysisPVLen = 15

# Report winrates for chat and analysis as (BLACK|WHITE|SIDETOMOVE).
# Default is SIDETOMOVE, which is what tools that use LZ probably also expect
# reportAnalysisWinratesAs = SIDETOMOVE

# Uncomment and and set to a positive value to make KataGo explore the top move(s) less deeply and accurately,
# but explore and give evaluations to a greater variety of moves, for analysis (does NOT affect play).
# A value like 0.03 or 0.06 will give various mildly but still noticeably wider searches.
# An extreme value like 1 will distribute many playouts across every move on the board, even very bad moves.
# analysisWideRootNoise = 0.0


# Default rules------------------------------------------------------------------------------------
# See https://lightvector.github.io/KataGo/rules.html for a description of the rules.
# These rules are defaults and can be changed mid-run by several custom GTP commands.
# See https://github.com/lightvector/KataGo/blob/master/docs/GTP_Extensions.md for those commands.

# Some other legal values are: "chinese", "japanese", "korean", "aga", "chinese-ogs", "new-zealand".
# KataGo does not claim to exactly match any particular human ruleset, but KataGo will try to behave
# as closely as possible given the rules it has implemented.
rules = japanese

# Use the below instead to specify an arbitrary combination of individual rules.

# koRule = SIMPLE       # Simple ko rules (triple ko = no result)
# koRule = POSITIONAL   # Positional superko
# koRule = SITUATIONAL  # Situational superko

# scoringRule = AREA       # Area scoring
# scoringRule = TERRITORY  # Territory scoring (uses a sort of special computer-friendly territory ruleset)

# taxRule = NONE  # All surrounded empty points are scored
# taxRule = SEKI  # Eyes in seki do NOT count as points
# taxRule = ALL   # All groups are taxed up to 2 points for the two eyes needed to live

# multiStoneSuicideLegal = true  #Is multiple-stone suicide legal? (Single-stone suicide is always illegal).

# hasButton = false # Set to true when area scoring to award 0.5 points to the first pass.

# whiteHandicapBonus = 0    # In handicap games, give white no compensation for black's handicap stones (Tromp-taylor, NZ, JP)
# whiteHandicapBonus = N-1  # In handicap games, give white N-1 points for black's handicap stones (AGA)
# whiteHandicapBonus = N    # In handicap games, give white N points for black's handicap stones (Chinese)

# Uncomment and change to adjust what board size KataGo uses upon startup by default if GTP doesn't specify.
# defaultBoardSize = 19

# Bot behavior---------------------------------------------------------------------------------------

# Resignation -------------

# Resignation occurs if for at least resignConsecTurns in a row,
# the winLossUtility (which is on a [-1,1] scale) is below resignThreshold.
allowResignation = true
resignThreshold = -0.90
resignConsecTurns = 3
# Uncomment to make katago not resign close games, behind by fewer than this many points
# resignMinScoreDifference = 10

# Handicap -------------

# Assume that if black makes many moves in a row right at the start of the game, then the game is a handicap game.
# This is necessary on some servers and for some GUIs and also when initializing from many SGF files, which may
# set up a handicap game using repeated GTP "play" commands for black rather than GTP "place_free_handicap" commands.
# However, it may also lead to incorrect understanding of komi if whiteHandicapBonus is used and a server does NOT
# have such a practice.
# Defaults to true! Uncomment and set to false to disable this behavior.
# assumeMultipleStartingBlackMovesAreHandicap = true

# Makes katago dynamically adjust in handicap or altered-komi games to assume based on those game settings that it
# must be stronger or weaker than the opponent and to play accordingly. Greatly improves handicap
# strength by biasing winrates and scores to favor appropriate safe/aggressive play.
# Does NOT affect analysis (lz-analyze, kata-analyze, used by programs like Lizzie) so analysis remains unbiased.
# Uncomment and set this to 0 to disable this and make KataGo play the same always.
# dynamicPlayoutDoublingAdvantageCapPerOppLead = 0.045

# Instead of a dynamic level, you can uncomment this and set this to a value from -3.0 to 3.0 to set KataGo's aggression to a FIXED level.
# DOES affect analysis tools (lz-analyze, kata-analyze, used by programs like Lizzie).
# Negative makes KataGo behave as if it is much weaker than the opponent, preferring to play defensively.
# Positive makes KataGo behave as if it is much stronger than the opponent, prefering to play aggressively or even overplay slightly.
# If this and "dynamicPlayoutDoublingAdvantageCapPerOppLead" are BOTH set then dynamic will be used for all games and this fixed
# value will be used for analysis tools.
# playoutDoublingAdvantage = 0.0

# Uncommenting one of these will enforce that the FIXED playoutDoublingAdvantage will only apply when KataGo plays the specified color
# and will be negated when playing the opposite color.
# playoutDoublingAdvantagePla = BLACK
# playoutDoublingAdvantagePla = WHITE

# Passing and cleanup -------------

# Make the bot never assume that its pass will end the game, even if passing would end and "win" under Tromp-Taylor rules.
# Usually this is a good idea when using it for analysis or playing on servers where scoring may be implemented non-tromp-taylorly.
# Defaults to true! Uncomment and set to false to disable this.
# conservativePass = true

# When using territory scoring, self-play games continue beyond two passes with special cleanup
# rules that may be confusing for human players. This option prevents the special cleanup phases from being
# reachable when using the bot for GTP play.
# Defaults to true! Uncomment and set to false if you want KataGo to be able to enter special cleanup.
# For example, if you are testing it against itself, or against another bot that has precisely implemented the rules
# documented at https://lightvector.github.io/KataGo/rules.html
# preventCleanupPhase = true

# Misc Behavior --------------------

# Uncomment and set to true to make KataGo avoid a particular joseki that some KataGo nets misevaluate,
# and also to improve opening diversity versus some particular other bots that like to play it all the time.
# avoidMYTDaggerHack = false

# Search limits-----------------------------------------------------------------------------------

# For all of "maxVisits", "maxPlayouts", "maxTime", search will still try to follow GTP time controls and may make a move
# faster than the specified max if GTP tells it that it is playing under a clock as well in the current game.

# If provided, limit maximum number of root visits per search to this much. (With tree reuse, visits do count earlier search)
maxVisits = 10000
# If provided, limit maximum number of new playouts per search to this much. (With tree reuse, playouts do not count earlier search)
# maxPlayouts = 300
# If provided, cap search time at this many seconds.
# maxTime = 10

# Ponder on the opponent's turn?
ponderingEnabled = true
maxTimePondering = 60  # Maximum time to ponder, in seconds. Comment out to make unlimited.
# Note: you can set "maxVisitsPondering" or "maxPlayoutsPondering" too.

# Number of seconds to buffer for lag for GTP time controls - will move a bit faster assuming there is this much lag per move.
lagBuffer = 1.0

# Number of threads to use in search
numSearchThreads = 12

# Play a little faster if the opponent is passing, for friendliness
searchFactorAfterOnePass = 0.50
searchFactorAfterTwoPass = 0.25
# Play a little faster if super-winning, for friendliness
searchFactorWhenWinning = 0.40
searchFactorWhenWinningThreshold = 0.95

# GPU Settings-------------------------------------------------------------------------------

# Maximum number of positions to send to a single GPU at once.
# The default value here is roughly equal to numSearchThreads, but you can specify it manually
# if you are running out of memory, or if you are using multiple GPUs that expect to split
# up the work.
# nnMaxBatchSize = <integer>

# Cache up to (2 ** this) many neural net evaluations in case of transpositions in the tree.
# Uncomment and edit to change if you want to adjust a major component of KataGo's RAM usage.
# nnCacheSizePowerOfTwo = 20

# Size of mutex pool for nnCache is (2 ** this).
# nnMutexPoolSizePowerOfTwo = 16

# Randomize board orientation when running neural net evals? Uncomment and set to false to disable.
# nnRandomize = true
# If provided, force usage of a specific seed for nnRandomize instead of randomizing.
# nnRandSeed = abcdefg

# TO USE MULTIPLE GPUS:
# Set this to the number of GPUs you have and/or would like to use.
# **AND** if it is more than 1, uncomment the appropriate CUDA or OpenCL section below.
# numNNServerThreadsPerModel = 1

# CUDA GPU settings--------------------------------------
# These only apply when using the CUDA version of KataGo.

# IF USING ONE GPU: optionally uncomment and change this if the GPU you want to use turns out to be not device 0
# cudaDeviceToUse = 0

# IF USING TWO GPUS: Uncomment these two lines (AND set numNNServerThreadsPerModel above):
# cudaDeviceToUseThread0 = 0  # change this if the first GPU you want to use turns out to be not device 0
# cudaDeviceToUseThread1 = 1  # change this if the second GPU you want to use turns out to be not device 1

# IF USING THREE GPUS: Uncomment these three lines (AND set numNNServerThreadsPerModel above):
# cudaDeviceToUseThread0 = 0  # change this if the first GPU you want to use turns out to be not device 0
# cudaDeviceToUseThread1 = 1  # change this if the second GPU you want to use turns out to be not device 1
# cudaDeviceToUseThread2 = 2  # change this if the third GPU you want to use turns out to be not device 2

# You can probably guess the pattern if you have four, five, etc. GPUs.

# KataGo will automatically use FP16 or not based on the compute capability of your NVIDIA GPU. If you
# want to try to force a particular behavior though you can uncomment these lines and change them
# to "true" or "false". E.g. it's using FP16 but on your card that's giving an error, or it's not using
# FP16 but you think it should.
# cudaUseFP16 = auto
# cudaUseNHWC = auto


# OpenCL GPU settings--------------------------------------
# These only apply when using the OpenCL version of KataGo.

# Uncomment to tune OpenCL for every board size separately, rather than only the largest possible size
# openclReTunePerBoardSize = true

# IF USING ONE GPU: optionally uncomment and change this if the best device to use is guessed incorrectly.
# The default behavior tries to guess the 'best' GPU or device on your system to use, usually it will be a good guess.
# openclDeviceToUse = 0

# IF USING TWO GPUS: Uncomment these two lines and replace X and Y with the device ids of the devices you want to use.
# It might NOT be 0 and 1, some computers will have many OpenCL devices. You can see what the devices are when
# KataGo starts up - it should print or log all the devices it finds.
# (AND also set numNNServerThreadsPerModel above)
# openclDeviceToUseThread0 = X
# openclDeviceToUseThread1 = Y

# IF USING THREE GPUS: Uncomment these three lines and replace X and Y and Z with the device ids of the devices you want to use.
# It might NOT be 0 and 1 and 2, some computers will have many OpenCL devices. You can see what the devices are when
# KataGo starts up - it should print or log all the devices it finds.
# (AND also set numNNServerThreadsPerModel above)
# openclDeviceToUseThread0 = X
# openclDeviceToUseThread1 = Y
# openclDeviceToUseThread2 = Z

# You can probably guess the pattern if you have four, five, etc. GPUs.


# Root move selection and biases------------------------------------------------------------------------------
# Uncomment and edit any of the below values to change them from their default.

# If provided, force usage of a specific seed for various things in the search instead of randomizing
# searchRandSeed = hijklmn

# Temperature for the early game, randomize between chosen moves with this temperature
# chosenMoveTemperatureEarly = 0.5
# Decay temperature for the early game by 0.5 every this many moves, scaled with board size.
# chosenMoveTemperatureHalflife = 19
# At the end of search after the early game, randomize between chosen moves with this temperature
# chosenMoveTemperature = 0.10
# Subtract this many visits from each move prior to applying chosenMoveTemperature
# (unless all moves have too few visits) to downweight unlikely moves
# chosenMoveSubtract = 0
# The same as chosenMoveSubtract but only prunes moves that fall below the threshold, does not affect moves above
# chosenMovePrune = 1

# Number of symmetries to sample (WITH replacement) and average at the root
# rootNumSymmetriesToSample = 1

# Using LCB for move selection?
# useLcbForSelection = true
# How many stdevs a move needs to be better than another for LCB selection
# lcbStdevs = 5.0
# Only use LCB override when a move has this proportion of visits as the top move
# minVisitPropForLCB = 0.15

# Internal params------------------------------------------------------------------------------
# Uncomment and edit any of the below values to change them from their default.

# Scales the utility of winning/losing
# winLossUtilityFactor = 1.0
# Scales the utility for trying to maximize score
# staticScoreUtilityFactor = 0.10
# dynamicScoreUtilityFactor = 0.30
# Adjust dynamic score center this proportion of the way towards zero, capped at a reasonable amount.
# dynamicScoreCenterZeroWeight = 0.20
# dynamicScoreCenterScale = 0.75
# The utility of getting a "no result" due to triple ko or other long cycle in non-superko rulesets (-1 to 1)
# noResultUtilityForWhite = 0.0
# The number of wins that a draw counts as, for white. (0 to 1)
# drawEquivalentWinsForWhite = 0.5

# Exploration constant for mcts
# cpuctExploration = 0.9
# cpuctExplorationLog = 0.4
# FPU reduction constant for mcts
# fpuReductionMax = 0.2
# rootFpuReductionMax = 0.1
# Use parent average value for fpu base point instead of point value net estimate
# fpuUseParentAverage = true
# Amount to apply a downweighting of children with very bad values relative to good ones
# valueWeightExponent = 0.5
# Slight incentive for the bot to behave human-like with regard to passing at the end, filling the dame,
# not wasting time playing in its own territory, etc, and not play moves that are equivalent in terms of
# points but a bit more unfriendly to humans.
# rootEndingBonusPoints = 0.5
# Make the bot prune useless moves that are just prolonging the game to avoid losing yet
# rootPruneUselessMoves = true

# How big to make the mutex pool for search synchronization
# mutexPoolSize = 16384
# How many virtual losses to add when a thread descends through a node
# numVirtualLossesPerThread = 1